In [ ]:
import pandas as pd
import re
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
%matplotlib inline
import string

import nltk
from nltk.corpus import stopwords

import spacy
from spacy.tokenizer import Tokenizer

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from wordcloud import WordCloud

In [2]:
df = pd.read_csv('cleaned.csv')
skills_df = pd.read_csv('skills.csv') 

In [3]:
url = 'http://127.0.0.1:5277/role/get_all'

In [4]:
res = urlopen(url).read()
content = json.loads(res)['content']
roles = []

for text in content: 
    roles.append(text['name'])

In [5]:
rows_to_keep = []

for role in roles:  
    found = df['jobtitle'].notna() & df.jobtitle.str.contains(role, case=False)
    if role == "UI/UX Desiigner":
        found = df['jobtitle'].notna() & (df.jobtitle.str.contains("UI Designer", case=False) | df.jobtitle.str.contains("UX Designer", case=False))
        
    df.loc[found, 'jobtitle'] = role
    
    temp = df[df.jobtitle.str.contains(role, case=False) == True].index 
    print(role, len(temp))
    for idx in temp: 
        if idx not in rows_to_keep: 
            rows_to_keep.append(idx)

Business Analyst 583
Data Analyst 161
Data Engineer 98
Pre-Sales Consultant 0
Security Engineer 216
Software Engineer 842
UI/UX Desiigner 119


In [6]:
df = df.loc[rows_to_keep]
df.reset_index(drop=True, inplace=True)

In [11]:
df

,company,jobtitle,jobdescription,skills
0,Genesis10,Business Analyst,Genesis10 is looking for a Business Analyst fo...,"Analysis, Analyst, Application, Business Analy..."
1,Genesis10,Business Analyst,Genesis10's G10 Associates Program is seeking ...,"Analyst, Business Analyst, Business Requiremen..."
2,"VanderHouwen & Associates, Inc.",Business Analyst,PLEASE JOIN OUR TALENT NETWORK: http://bit.ly/...,SQL Reporting Services Suite
3,"VanderHouwen & Associates, Inc.",Business Analyst,VanderHouwen has more jobs you may like! Find ...,Telecommuting not available Travel not required
4,SMCI,Business Analyst,Job Description: Seeking a Business Process An...,"Mortgage/Equiting Lending, Lean Six Sigma Gree..."
...,...,...,...,...
2014,"Creative Circle, LLC",UI/UX Desiigner,Position: UX Designer (Agency / Axure skills n...,"Axure, Photoshop, InDesign & Illustrator"
2015,Combined Computer Resources,UI/UX Desiigner,LOOKING FOR A UX DESIGNER TO WORK IN THE CUSTO...,UX FRAMEWORKS FRONT-END DEV SKETCH VISION
2016,"K2 Partnering Solutions, Inc.",UI/UX Desiigner,My Direct Client is looking for a UX Designer ...,"web design, wireframe, wireframming, design, r..."
2017,"OSI Engineering, Inc.",UI/UX Desiigner,The ideal candidate will have experience desig...,"UI UX ""user experience"" gui ""user interface"" g..."


In [ ]:
## preprocessing

stop_words = set(stopwords.words('english'))
additional_stopwords = ['junior', 'senior','experience','etc','job','work','company','technique',
                    'candidate','skill','skills','language','menu','inc','new','plus','years',
                   'technology','organization','ceo','cto','account','manager','data','scientist','mobile',
                    'developer','product','revenue','strong', 'big']

def remove_stopwords(sentence):
    return ' '.join([word for word in sentence.split() if not word in stop_words])

def remove_add_stopwords(sentence):
    return ' '.join([word for word in sentence.split() if not word in additional_stopwords])

def sentence_preprocessing(df, col):
    df[col] = df[col].str.lower() # convert to lowercase
    df[col] = df[col].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ' , x)) # remove puncutations
    df[col] = df[col].apply(lambda x : re.sub(r"\d+", " ", x)) # remove numbers
    df[col] = df[col].apply(remove_stopwords)
    df[col] = df[col].apply(remove_add_stopwords)
    return df[col]

df['jobdescription'] = sentence_preprocessing(df, 'jobdescription')
df['skills'] = sentence_preprocessing(df, 'skills')

In [ ]:
Business_Analyst_coutner = {}
Data_Analyst_coutner = {}
Data_Engineer_coutner = {}
Security_Engineer_coutner = {}
Software_Engineer_coutner = {}

for i in range(len(df)): 
    title = df['jobtitle'][i]
    desc = df['jobdescription'][i]
    skills = df['skills'][i]
    for skill in skills_df['preferredLabel']: 
        if skill in desc and skill in Business_Analyst_coutner and title == 'Business Analyst': 
            Business_Analyst_coutner[skill] += 1
        elif skill in desc and skill not in Business_Analyst_coutner and title == 'Business Analyst':
            Business_Analyst_coutner[skill] = 1
            
        elif skill in desc and skill in Data_Analyst_coutner and title == 'Data Analyst': 
            Data_Analyst_coutner[skill] += 1
        elif skill in desc and skill not in Data_Analyst_coutner and title == 'Data Analyst':
            Data_Analyst_coutner[skill] = 1
            
        elif skill in desc and skill in Data_Engineer_coutner and title == 'Data Engineer': 
            Data_Engineer_coutner[skill] += 1
        elif skill in desc and skill not in Data_Engineer_coutner and title == 'Data Engineer':
            Data_Engineer_coutner[skill] = 1
            
        elif skill in desc and skill in Security_Engineer_coutner and title == 'Security Engineer': 
            Security_Engineer_coutner[skill] += 1
        elif skill in desc and skill not in Security_Engineer_coutner and title == 'Security Engineer':
            Security_Engineer_coutner[skill] = 1
        
        elif skill in desc and skill in Software_Engineer_coutner and title == 'Software_Engineer': 
            Software_Engineer_coutner[skill] += 1
        elif skill in desc and skill not in Software_Engineer_coutner and title == 'Software_Engineer':
            Software_Engineer_coutner[skill] = 1

In [ ]:
Business_Analyst_coutner

In [ ]:
Data_Analyst_coutner

In [ ]:
Data_Engineer_coutner

In [ ]:
Security_Engineer_coutner

In [ ]:
Software_Engineer_coutner

In [ ]:
# test = df.groupby(['jobtitle']).sum().reset_index()
# for role in test.jobtitle.unique().tolist(): 
#     text = test[test.jobtitle == role].iloc[0].jobdescription
#     wordcloud = WordCloud().generate(text)
#     print(role)
#     plt.imshow(wordcloud, interpolation='bilinear')
#     plt.axis("off")
#     plt.show()

In [ ]:
## tokenization

# nlp = spacy.load('en_core_web_lg')
# spacytokenizer = Tokenizer(nlp.vocab)

# df['jobdescription'] = df['jobdescription'].apply(lambda x: [ token.text for token in spacytokenizer(x)])
# df['skills'] = df['skills'].apply(lambda x: [ token.text for token in spacytokenizer(x)])

In [ ]:
# def get_top_n_grams(bag_of_words, vec, n):
#     sum_words = bag_of_words.sum(axis=0) 
#     words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
#     words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
#     return words_freq[:n]

# df['jobdescription'] = [' '.join(map(str, l)) for l in df['jobdescription']]
# df['skills'] = [' '.join(map(str, l)) for l in df['skills']]

In [ ]:
# X = df['jobdescription']
# y = df['skills']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df.jobdescription)
y = df.skills

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=42)

In [ ]:
from sklearn import metrics
from sklearn import svm

svm_classifier = svm.SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

svm_classifier.fit(X_train, y_train)

# Predict the labels on the test dataset
svm_predictions_tf = svm_classifier.predict(X_test)

In [ ]:
print("Accuracy score is: ",accuracy_score(y_test, svm_predictions_tf))
print("Classes: (to help read Confusion Matrix)\n", svm_classifier.classes_)
print("Confusion Matrix: ")

print(confusion_matrix(y_test, svm_predictions_tf))
print("Classification Report: ")
print(classification_report(y_test, svm_predictions_tf))

In [ ]:
support_vectors = svm_classifier.support_vectors_

# Get dual coefficients (Lagrange multipliers)
dual_coefficients = svm_classifier.dual_coef_.toarray().flatten()

# Initialize an array to store feature importance
feature_importance = [0] * len(vectorizer.get_feature_names_out())

# Accumulate feature importance based on support vectors and dual coefficients
for idx, dual_coef in zip(svm_classifier.support_, dual_coefficients):
    feature_importance += dual_coef * X_train[idx].toarray().flatten()

# Combine feature names and accumulated importance into a dictionary
feature_dict = dict(zip(vectorizer.get_feature_names_out(), feature_importance))

# Sort features by their accumulated importance
sorted_features = sorted(feature_dict.items(), key=lambda x: abs(x[1]), reverse=True)

# Display the most representative words/tokens
top_words = [word for word, _ in sorted_features[:10]]  # Adjust the number as needed
print("Top representative words/tokens:")
print(top_words)

In [ ]:
new_descriptions = ["Exciting opportunity for a software engineer with Python skills.",
                    "Data analyst position available for candidates with strong analytical skills."]

# Preprocess and vectorize new descriptions
new_descriptions_tfidf = vectorizer.transform(new_descriptions)

# Predict skills
predicted_skills = svm_classifier.predict(new_descriptions_tfidf)

# Display the predicted skills
print("Predicted Skills:")
print(predicted_skills)

In [ ]:
df

In [7]:
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

In [8]:
nlp = spacy.load("en_core_web_lg")
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [18]:
string = '''About OKX Graduate Program (Supernova)
The Supernova Program is a 3-year Career Accelerator Program that aims to fast-track young, high performing graduates into technical experts and future leaders mainly in the fields of Product Engineering, Product Management, and Product Design. We firmly believe in the youthful power of the new era. Join us to achieve your narrative around crypto.
 
As a graduate Software Engineer, you will build and maintain the OKX platform with millions of daily active users. You will work cross-functionally with design, product, and other engineering teams to identify customer needs, and ship high-quality new features through fast iterations. This is an opportunity to learn the full life cycle of crypto platforms including pro/retail trading, asset management, and wallet.
 
Hiring Teams
Mobile: As a Mobile Engineer, you will build and maintain our core mobile app with millions of daily active users. You will have the opportunity to learn the full life cycle of a crypto mobile application including retail/pro trading, asset management, wallet and more.
Frontend: As a Frontend Engineer, you will be working closely with the top-notch frontend development team on a wide variety of products - developing with performance in mind to serve a world class digital asset trading platform with millions of active users.
Backend: As a Backend Engineer, you will focus on building our state-of-the-art Cefi/Defi/Quant platform to help our global customers buy and sell their digital assets seamlessly and flawlessly.
What You’ll Be Doing
Be part of the global engineering team to design and develop cutting-edge crypto-linked products on the platform. Deliver customized solutions with performance, scalability and continuous integration in mind while maintaining overall architecture consistency.
Be a problem solver to build high availability, high performance distributed service. Strive for optimization and maximize user satisfaction in a time-critical manner.
Partner closely with Design, Product and Marketing teams, drive rapid innovation by turning design ideas into quality features that scale to millions of users. Communicate effectively to peers, cross-functions and leadership on project deliverables.
Stay on top of industry trends, and evaluate new solutions to address our unique and fast-growing business needs. Prototype ideas quickly using cutting edge technologies.
Maintain high-quality technical documentation. Uphold technology best practices and code reviews with peers. Improve efficiency in cross-office/time zone collaboration.
What We Look For In You
Bachelors in Computer Science, Technology, Engineering, Mathematics, related technical disciplines, or self-taught enthusiasts
Possess relevant tech stack skillset for the respective specialisation - Java/SpringBoot/SpringCloud, Microservice/API development, JavaScript, React/Angular/Vue, HTML5/CSS3, Kotlin/Objective-C/Swift, relational databases, data structures and algorithms.
Passionate about the latest technology. Curious to learn all aspects of trading products in financial and crypto markets, including spot, future, swap, options, and structured products.
Analytical with positive problem-solving mindset, A pro-active team player that embodies a growth mindset, flexible and comfortable in navigating ambiguity with a global mindset
Experience in practicing automated testing and continuous integration
Nice to Haves
Experience in internship/ freelance projects, designed and developed micro-service programs, or capability for cross-browser development.
Comfortable with the cloud-based Linux environment. Knowledgeable in multi-threading and distributed architecture. Understanding of mainstream messaging frameworks including Kafka. Or familiar with daily developing tools such as npm, gulp, webpack, git, etc.
Interested in equipping themselves to be full-stack architects and open to rotate amongst specialisation. Curious and excited in crypto/ block chain industry.'''

annotations = skill_extractor.annotate(string)

C:\Users\lqlau\AppData\Roaming\Python\Python39\site-packages\skillNer\utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


In [19]:
skill_extractor.describe(annotations)

In [25]:
# for i in range(len(annotations['results'])): 
#     print()

annotations['results']['full_matches'][0]['doc_node_value']

'product engineering'

In [14]:
df_new = df.groupby(["jobtitle"])['jobdescription'].sum().reset_index()

In [17]:
df_new['jobdescription'][0]

'Genesis10 is looking for a Business Analyst for a long-term contract position in Sandy Springs, GA with our Fortune25 financial services industry client. Only W2 candidates will be considered. Description:  As a Business Analyst, you will conduct business and technical analysis for process and tool enhancements and ensures operational readiness to satisfy business partner objectives. The Business Analyst will employ a set of tasks and techniques used to work as a liaison among stakeholders and core team members in order to understand the structure, policies and operations of an organization, and to recommend solutions that enable the organization to achieve its goals. Responsibilities:  * Business Analysis, Planning and Monitoring * Requirements Management and Communication * Enterprise Analysis * Requirements Analysis * Solution Assessment and Validation * Work with core team members to elicit, organize, document and validate business rules, functional and non-functional requirements

In [36]:
counts_dic = {}

for i in range(len(df_new.jobdescription)):
    temp_dic = {}
    annotation = skill_extractor.annotate(string)
    for key in annotations['results'].keys(): 
        temp = annotations['results'][key]
        for j in range(len(temp)): 
            term = temp[i]['doc_node_value']
            if term in temp_dic: 
                temp_dic[term] += 1
            else: 
                temp_dic[term] = 1
    counts_dic[df_new.jobtitle[i]] = temp_dic


In [38]:
counts_dic

{'Business Analyst': {'product engineering': 18, 'supernova': 60},
 'Data Analyst': {'product management': 18, 'supernova': 60},
 'Data Engineer': {'product design': 18, 'track': 60},
 'Security Engineer': {'asset management': 18, 'software engineer': 60},
 'Software Engineer': {'mobile app': 18, 'maintain': 60},
 'UI/UX Desiigner': {'asset management': 18, 'active': 60}}

In [40]:
annotation['results'][]


{'full_matches': [{'skill_id': 'KS1282K6XPGKSGR1826F',
   'doc_node_value': 'product engineering',
   'score': 1,
   'doc_node_id': [35, 36]},
  {'skill_id': 'KS1282M68WL9T4YH3SLF',
   'doc_node_value': 'product management',
   'score': 1,
   'doc_node_id': [37, 38]},
  {'skill_id': 'KS1282K6QK68NJHMG8V2',
   'doc_node_value': 'product design',
   'score': 1,
   'doc_node_id': [40, 41]},
  {'skill_id': 'KS120PL63HT8CXQ6LPDR',
   'doc_node_value': 'asset management',
   'score': 1,
   'doc_node_id': [122, 123]},
  {'skill_id': 'KS440NM6N05DM8TSV6QZ',
   'doc_node_value': 'mobile app',
   'score': 1,
   'doc_node_id': [140, 141]},
  {'skill_id': 'KS120PL63HT8CXQ6LPDR',
   'doc_node_value': 'asset management',
   'score': 1,
   'doc_node_id': [168, 169]},
  {'skill_id': 'KS122Y2690XG35MVZ4N1',
   'doc_node_value': 'digital asset',
   'score': 1,
   'doc_node_id': [206, 207]},
  {'skill_id': 'KS122Y2690XG35MVZ4N1',
   'doc_node_value': 'digital asset',
   'score': 1,
   'doc_node_id': [243